# Minimum Data Proficiency Guide
## UNC Street Drug Analysis Lab

---

This guide is a self-administered test of basic data functions and conventions in order to be able to use our results dataset accurately. The code below will walk you through a minimum analysis from a one of our public datasets. After each numbered question below, the code and correct results are provided for you to compare. **We strongly recommend that you write your own code in a separate editor and run the same exact analysis in paralell.** Completion of that code and results, in comparison to the results below, will constituent sufficient data proficiency to become a regular user of the data we produce.<br>

### Training Materials
Please review the [codebook](https://github.com/opioiddatalab/drugchecking/blob/main/datasets/unc_druchecking_codebook.txt) and [technical details](https://github.com/opioiddatalab/drugchecking/blob/main/datasets/technical_details.md) before starting this exercise. [This tutorial](https://www.dataquest.io/blog/jupyter-notebook-tutorial/) explains Jupyter notebooks if you've never used them before, but you can also copy the code into a text-based editor and run it from there.

---

### Operating Environment

Making it clear which versions of software we are using for posterity. Not part of the exercise.

In [1]:
display "Notebook generated at $S_TIME ET on$S_DATE"
display "Running Stata MP:"
version
display "On JupyterLab Version 4.2.5"


Notebook generated at 17:10:59 ET on 8 Oct 2024

Running Stata MP:

version 17.0

On JupyterLab Version 4.2.5


---

# 1. Load data
Results files `analysis_dataset.*` and `lab_detail.*` are provided in the following file formats:
+ Stata (v14 or later) - `*.dta`
+ Excel (XML) - `*.xlsx`
+ SAS (transport v8) - `*.v8xpt`
+ CSV (comma-separated with quote enclosure for strings) - `*.csv`

<br>Demo datasets used in this exercise can be [downloaded from our GitHub](https://github.com/opioiddatalab/drugchecking/tree/main/datasets) in the format of your choice. Below we will call them directly, using the "Raw" version within the GitHub interface.<br>

We will load `analysis_dataset.dta` into a frame called `card`. And `lab_detail.dta` will go into a frame called `lab`.

In [2]:
// Reset environment
clear all
frames reset

// Load lab dataframe
frame create lab
frame change lab
use https://github.com/opioiddatalab/drugchecking/raw/refs/heads/main/datasets/lab_detail.dta
display "LAB data file variables:"
describe, simple

// Load card dataframe
frame create card
frame change card
use https://github.com/opioiddatalab/drugchecking/raw/refs/heads/main/datasets/analysis_dataset.dta
display "CARD data file variables:"
describe, simple







LAB data file variables:

sampleid      lab_num_su~y  lab_co~e_any  lab_btmps     lab_nitaze~y
substance     lab_num_su~s  lab_caffeine  chemdict      lab_meth_i~y
pubchemcid    confirmatory  lab_gabape~n  lab_opioid    lab_mdma_i~y
cas           lab_fentanyl  lab_levami~y  lab_opioid~y  lab_c~es_any
unii          lab_fe~l_any  lab_mdma      lab_synthe~d  lab_common~y
abundance     lab_xylazine  lab_tramadol  lab_synthe~y  lab_heroin~y
method        lab_xylazi~y  lab_carfen~l  lab_potent~e  lab_cannab~y
date_compl~e  lab_xylaz~ce  lab_carfen~y  lab_potent~y  lab_fe~s_any
primary       lab_meth      lab_ketami~y  lab_benzod~e  lab_pf_fen~y
trace         lab_meth_any  lab_ketamine  lab_benzod~y  lab_substi~y
lab_null      lab_cocaine   lab_btmps_~y  lab_nitazene




CARD data file variables:

sampleid      bright_color  overdose      lab_gabape~n  lab_mdma_i~y
program       texture       od            lab_levami~y  lab_c~es_any
location      tar           overdose_n~s  lab_mdma    

Variables may have evolved since this was run, but the first ones should be the same and the general pattern of `lab_`, `expect_`, `sen_`, etc. should hold. At this point we suggest opening up a grid-style variable table/explorer and getting to know the variable formats using the [codebook](https://github.com/opioiddatalab/drugchecking/blob/main/datasets/unc_druchecking_codebook.txt).

---

# 2. How many unique samples? 

In [3]:
display "Unique samples in CARD:"
distinct sampleid


Unique samples in CARD:


          |        Observations
          |      total   distinct
----------+----------------------
 sampleid |         20         20


There are 20 rows and 20 unique `sampleid` in `CARD`. One row per sample.<br>
(If you don't have [-distinct-](https://www.stata.com/statalist/archive/2012-03/msg00893.html) you can SSC install.)<br>
<br>
Now, let's repeat this for `LAB`.

In [4]:
frame change lab
display "Unique samples in LAB:"
distinct sampleid



Unique samples in LAB:


          |        Observations
          |      total   distinct
----------+----------------------
 sampleid |        115         20


So here we see there are 115 rows for 20 unique samples. Each row in LAB is a separate substance identified, so multiple rows per sample unless only one substance was detected in a given sample.

---

# 3. How many and which substances detected?
Staying in `LAB` we run the same command on column `substance`.

In [5]:
display "Unique substances detected:"
distinct substance


Unique substances detected:


           |        Observations
           |      total   distinct
-----------+----------------------
 substance |        115         47


So among the 20 samples there were 47 unqiue substances detected. Let's make a table of them.

In [6]:
tabulate substance, sort


   Confirmed substance detected in lab. |
            Standardized chemical name. |      Freq.     Percent        Cum.
----------------------------------------+-----------------------------------
                               fentanyl |         11        9.57        9.57
                                 4-ANPP |          9        7.83       17.39
                        methamphetamine |          7        6.09       23.48
                               xylazine |          5        4.35       27.83
                           1,3-Diacetin |          4        3.48       31.30
                                cocaine |          4        3.48       34.78
                                 heroin |          4        3.48       38.26
                       p-fluorofentanyl |          4        3.48       41.74
                       phenethyl 4-ANPP |          4        3.48       45.22
                          acetylcodeine |          3        2.61       47.83
                               ca

---

# 4. Where and how were samples collected?
Let's switch to `CARD` because this is likely where you'll be doing most epi analyses. We want to know more about the 20 samples, where they came from, how they were collected.

In [7]:
frame change card
tabulate state, sort missing
distinct state




 State sample |
    collected |      Freq.     Percent        Cum.
--------------+-----------------------------------
           NC |          8       40.00       40.00
           NY |          4       20.00       60.00
           OR |          2       10.00       70.00
           TN |          2       10.00       80.00
           MI |          1        5.00       85.00
           MT |          1        5.00       90.00
           NM |          1        5.00       95.00
           TX |          1        5.00      100.00
--------------+-----------------------------------
        Total |         20      100.00


       |        Observations
       |      total   distinct
-------+----------------------
 state |         20          8


Now we know the 20 samples came from 8 different states. Of the n=8 samples from North Carolina, let's find out which counties they originated in.

In [8]:
tabulate county if state=="NC", sort missing


          County where sample collected |      Freq.     Percent        Cum.
----------------------------------------+-----------------------------------
                        Guilford County |          2       25.00       25.00
                        Buncombe County |          1       12.50       37.50
                         Catawba County |          1       12.50       50.00
                          Gaston County |          1       12.50       62.50
                            Pitt County |          1       12.50       75.00
                        Randolph County |          1       12.50       87.50
                            Wake County |          1       12.50      100.00
----------------------------------------+-----------------------------------
                                  Total |          8      100.00


If you're interested in making maps you'll find the county 5-digit FIPS codes handy. You don't have to do this next code block as part of the exercise, but you can see the variables below. Please see the codebook documentation on how we assign county FIPS codes using an API. It can make mistakes, so if you see a wayward county, please let us know.

In [9]:
list county state countyfips


     +-------------------------------------+
     |           county   state   county~s |
     |-------------------------------------|
  1. |  Kalkaska County      MI      26079 |
  2. |  Missoula County      MT      30063 |
  3. |  Buncombe County      NC      37021 |
  4. |   Catawba County      NC      37035 |
  5. |    Gaston County      NC      37071 |
     |-------------------------------------|
  6. |  Guilford County      NC      37081 |
  7. |  Guilford County      NC      37081 |
  8. |      Pitt County      NC      37147 |
  9. |  Randolph County      NC      37151 |
 10. |      Wake County      NC      37183 |
     |-------------------------------------|
 11. |  Santa Fe County      NM      35049 |
 12. |     Kings County      NY      36047 |
 13. |     Kings County      NY      36047 |
 14. |  Saratoga County      NY      36091 |
 15. |   Suffolk County      NY      36103 |
     |-------------------------------------|
 16. | Multnomah County      OR      41051 |
 17. | Mu

In [32]:
graph display
graph hbar lab_num_substances_any, over(state)



file /Users/nabarun/.stata_kernel_cache/graph1.svg saved as SVG format
file /Users/nabarun/.stata_kernel_cache/graph1.pdf saved as PDF format

variable state not found



---

Next let's find out the sample collection methods.

In [11]:
tabulate sampletype, sort missing


 Verbatim sample |
    type on card |      Freq.     Percent        Cum.
-----------------+-----------------------------------
         unknown |          5       25.00       25.00
          cotton |          4       20.00       45.00
            swab |          4       20.00       65.00
         spatula |          3       15.00       80.00
            pill |          2       10.00       90.00
   swab; spatula |          1        5.00       95.00
         syringe |          1        5.00      100.00
-----------------+-----------------------------------
           Total |         20      100.00


We see the types of collection methods used in these samples. Since the verbatim `sampletype` pulls from whatever was circled on the card, we may want to consolidate all the possible combinations of multiple collection methods. So, we have created a variable that makes this easier to report called `collection`. Throughout the codebook, you'l find similar re-coded variables next to verbatim card data.

---

# 5. What sensations and ODs were reported?

First to make this more interesting, let's figure out how many samples contained fentanyl. The easiest way in `CARD` is to use a pre-made variable called `lab_fentanyl`.

In [12]:
tabulate sensations if lab_fentanyl==1, sort missing


             Sensations circled on card |      Freq.     Percent        Cum.
----------------------------------------+-----------------------------------
                                 normal |          3       30.00       30.00
                                        |          2       20.00       50.00
                                  weird |          2       20.00       70.00
                   nice; stronger; long |          1       10.00       80.00
                               stronger |          1       10.00       90.00
              weird; weaker; unpleasant |          1       10.00      100.00
----------------------------------------+-----------------------------------
                                  Total |         10      100.00


Because multiple sensations can be reported and circled on a card, the verbatim `sensations` can get complicated. (Also there is `sensation_notes` to consider.) So we have recoded sensation variables that you'll find in the codebook as variables with the prefix `sen_`. Let's look at `sen_strength` which is a 3-level categorical variable where `-1` is weaker, `0` is normal, and `1` is stronger.

In [13]:
tabulate sen_strength if lab_fentanyl==1, sort missing nolabel


    Derived |
    3-level |
 flag where |
 -1 weaker, |
0 normal, 1 |
   stronger |      Freq.     Percent        Cum.
------------+-----------------------------------
          . |          4       40.00       40.00
          0 |          3       30.00       70.00
          1 |          2       20.00       90.00
         -1 |          1       10.00      100.00
------------+-----------------------------------
      Total |         10      100.00


The Stata datasets provide variable labeling support, which we suppress above with the `nolabel` option. Below is the tidier table with labels.

In [14]:
tabulate sen_strength if lab_fentanyl==1, sort missing


    Derived |
    3-level |
 flag where |
 -1 weaker, |
0 normal, 1 |
   stronger |      Freq.     Percent        Cum.
------------+-----------------------------------
          . |          4       40.00       40.00
     normal |          3       30.00       70.00
   stronger |          2       20.00       90.00
     weaker |          1       10.00      100.00
------------+-----------------------------------
      Total |         10      100.00


You can see the missingness in the original card data also carries through to the recoded variable. The more completely you fill out the card data, the better these data inferences will be! We find this a handy variable to give a quick sense for potency. It could even have application as a signal detection tool if suddenly stronger dope is being reported. 

---

Now to look at overdoses. A similar convention of verabtim (`overdose`) and recoded (`od`) for overdose are available. Code below for all 20 samples.

In [15]:
tabulate overdose, missing sort


      Invovlement in overdose |      Freq.     Percent        Cum.
------------------------------+-----------------------------------
         not overdose related |         15       75.00       75.00
               involved in OD |          3       15.00       90.00
                      unknown |          2       10.00      100.00
------------------------------+-----------------------------------
                        Total |         20      100.00


In [16]:
tabulate od, missing nolabel sort


    Recoded |
   overdose |
involvement |
        for |
  reporting |      Freq.     Percent        Cum.
------------+-----------------------------------
          0 |         15       75.00       75.00
          1 |          3       15.00       90.00
          . |          2       10.00      100.00
------------+-----------------------------------
      Total |         20      100.00


The `od` variable is therefore set up to be ready for the outcome/dependent variable in regression models if you are trying to find correlations with overdose risk. Here's a simple binomial regression to get a prevelance ratio (we don't use odds ratios and logistic regression because of assumption violations in this dataset and interpretability). This is an optional exercise; skip it if you're not familiar with these regression model forms.

In [17]:
binreg lab_fentanyl od, rr


Iteration 1:   deviance =  34.02278
Iteration 2:   deviance =  24.77439
Iteration 3:   deviance =  24.54743
Iteration 4:   deviance =  24.54678
Iteration 5:   deviance =  24.54678

Generalized linear models                         Number of obs   =         18
Optimization     : MQL Fisher scoring             Residual df     =         16
                   (IRLS EIM)                     Scale parameter =          1
Deviance         =  24.54678429                   (1/df) Deviance =   1.534174
Pearson          =  17.99995675                   (1/df) Pearson  =   1.124997

Variance function: V(u) = u*(1-u)                 [Bernoulli]
Link function    : g(u) = ln(u)                   [Log]

                                                  BIC             =  -21.69916

------------------------------------------------------------------------------
             |                 EIM
lab_fentanyl | Risk ratio   std. err.      z    P>|z|     [95% conf. interval]
-------------+----------------

Or if you prefer Poisson models, here's the same result.

In [18]:
poisson lab_fentanyl od , irr vce(robust)


Iteration 0:   log pseudolikelihood = -15.145911  
Iteration 1:   log pseudolikelihood = -15.145911  

Poisson regression                                      Number of obs =     18
                                                        Wald chi2(1)  =   0.49
                                                        Prob > chi2   = 0.4818
Log pseudolikelihood = -15.145911                       Pseudo R2     = 0.0061

------------------------------------------------------------------------------
             |               Robust
lab_fentanyl |        IRR   std. err.      z    P>|z|     [95% conf. interval]
-------------+----------------------------------------------------------------
          od |   1.428572    .724418     0.70   0.482     .5287667    3.859579
       _cons |   .4666667   .1325467    -2.68   0.007     .2674491    .8142774
------------------------------------------------------------------------------
Note: _cons estimates baseline incidence rate.


**Interpretation:** So an IRR of 1.43 translates to samples containing fentanyl were 43% more likely to be involved in a reported overdose, compared to samples that didn't contain fentanyl.

---

# 6. How many blue pills?

Say we want to get the sample IDs for all the blue pills. First we can make a variable called `blue` and then combine it with the pre-existing `pill` variable.

In [19]:
// Verbatim colors
tabulate color, missing

// Generate 'blue'
** regular expressions search
gen blue = regexm(color, "blue")
** account for missing card data
replace blue=. if color==""
** reorder for human readability
order blue, a(color)
** see results
list sampleid color blue



                  Color circled on card |      Freq.     Percent        Cum.
----------------------------------------+-----------------------------------
                                        |          1        5.00        5.00
                                  black |          1        5.00       10.00
                                   blue |          2       10.00       20.00
                                  brown |          2       10.00       30.00
                                  clear |          2       10.00       40.00
                           clear; white |          1        5.00       45.00
                            gray; brown |          1        5.00       50.00
                            light brown |          1        5.00       55.00
                             light gray |          1        5.00       60.00
                                    tan |          1        5.00       65.00
                                unknown |          1        5.00       70.

Coding looks good. There are two samples colored blue. Let's see if they were pills.

In [20]:
tabulate blue pill, missing


           | Composite flag (1) if
           |    indication that
           |   sample was from a
           |         pill
      blue |         1          . |     Total
-----------+----------------------+----------
         0 |         0         17 |        17 
         1 |         2          0 |         2 
         . |         0          1 |         1 
-----------+----------------------+----------
     Total |         2         18 |        20 


Not surprisingly the two blues were pills. Let's make a `blue_pill` variable.

In [21]:
gen blue_pill = 0
replace blue_pill = 1 if blue==1 & pill==1
replace blue_pill = . if color=="" // deal with missing color on card
list sampleid blue_pill color pill expectedsubstance if blue_pill==1, abbreviate(20) 
tabulate blue_pill, missing



(2 real changes made)

(1 real change made, 1 to missing)


     +---------------------------------------------------------+
     | sampleid   blue_pill   color   pill   expectedsubstance |
     |---------------------------------------------------------|
  6. |      280           1    blue      1                MDMA |
 16. |      836           1    blue      1            fentanyl |
     +---------------------------------------------------------+


  blue_pill |      Freq.     Percent        Cum.
------------+-----------------------------------
          0 |         17       85.00       85.00
          1 |          2       10.00       95.00
          . |          1        5.00      100.00
------------+-----------------------------------
      Total |         20      100.00


By bringing in the `expectedsubstance` we can see that of the two samples, one was a fake pill and and one an XTC tablet. Data from 2023 forward also have a specific M30 category for expected substance.

In [22]:
drop blue_pill

---

# 7. How many samples with acetylcodeine?

Say we don't have a pre-made `lab_` variable for a substance in `CARD` dataframe. We've seen how useful `CARD` is for epidemiology analysis because it is one-row-per-sample. Let's make one for acetylcodeine, a common weak opioid found with poppy-derived heroin. There are other ways to do make this variable, but this should get you started.<br>
<br>
But first, review the technical details at the top to understand the `_any` convention. In brief, the prefix `lab_` in a variable designates that the substance was positively identified in the lab using GCMS only in *primary* abundance. The `_any` suffix includes *both primary and trace* abundance.<br>
<br>
We use the term *trace* to mean small levels of substance detected that would not normally be reported out in a forensic chemistry report for a crime lab. However, for public health purposes, trace substances can be responsible for unexpected side effects and reactions, or may help uncover changes to synthesis methods, or may explain test strip results, etc. We define *trace* as less than 5% peak height area of the most abundant molecule on the GCMS chromatogram. It is ***not*** interpreted as <5% weight/volume. In the variable `abundance` the value "trace" is positively defined; if `abundance` is blank then the abundance is *primary*. <br>

In [23]:
frame change lab

// Create 2 new variables in LAB dataframe using lab_ prefix and _any suffix.

// Create variable for trace OR primary abundance with _any suffix
gen lab_acetylcodeine_any = regexm(lower(substance),"acetylcodeine")
la var lab_acetylcodeine_any "Acetylcodeine detected in primary or trace abundance"
tab lab_acetylcodeine_any, m

// Create variable for only PRIMARY abundance without any suffix
gen lab_acetylcodeine = 0
replace lab_acetylcodeine=1 if regexm(lower(substance),"acetylcodeine") & abundance==""
la var lab_acetylcodeine "Acetylcodeine detected in PRIMARY abundance only"
tab lab_acetylcodeine, m






Acetylcodei |
ne detected |
 in primary |
   or trace |
  abundance |      Freq.     Percent        Cum.
------------+-----------------------------------
          0 |        112       97.39       97.39
          1 |          3        2.61      100.00
------------+-----------------------------------
      Total |        115      100.00


(1 real change made)



Acetylcodei |
ne detected |
 in PRIMARY |
  abundance |
       only |      Freq.     Percent        Cum.
------------+-----------------------------------
          0 |        114       99.13       99.13
          1 |          1        0.87      100.00
------------+-----------------------------------
      Total |        115      100.00


Now we have 2 new variables in the `LAB` dataframe: `lab_acetylcodeine_any` and `lab_acetylcodeine`. You may now be able to see the logic of having two separate datasets with lab and card data. The `LAB` file allows us to create `lab_` variables as we want and have the chemical logic behind them be reproduceible. For example, if we wanted to create a combined variable for MDA and MDMA we could do that within `LAB` and then pull it into the `CARD` dataframe. For classes of molecules (chemical families) we use the same process, such as for benzos. This way you can always trace back exactly which substance in which abundance is gave rise to any `lab_` variable in `CARD`. **So, always make `lab_` variables only in `LAB` dataframe.**

In [24]:
// Take sampleid and 2 new variables into new dataframe called 'varadd'
frame put sampleid lab_acetylcodeine lab_acetylcodeine_any, into(varadd)
frame change varadd

// Since there are multiple rows per sample, and the new lab_ variables are 1/0,
// we take the max so that if any substance triggers a '1' value it is
// consolidated as '1' at the sample level. A duplicates check is helpful just in case.
// Collapse doesn't preseve var labels (annoying!) so add those back in.

collapse (max) lab_acetylcodeine lab_acetylcodeine_any, by(sampleid)
la var lab_acetylcodeine_any "Acetylcodeine detected in primary or trace abundance"
la var lab_acetylcodeine "Acetylcodeine detected in PRIMARY abundance only"
duplicates drop

// Save the new variables as a new file and merge it into the CARD dataframe.
// Delete the temporary file.
// There are also ways to do this with -fralias- in Stata 18 onwards, but this
// method below will work with all Stata versions.

save varadd, replace 
frame change card
merge 1:1 sampleid using varadd, nogen
erase varadd.dta

// Check results
display "lab_acetylcodeine:"
tabulate lab_acetylcodeine, missing
display "lab_acetylcodeine_any:"
tabulate lab_acetylcodeine_any, missing

tabulate lab_acetylcodeine_any lab_acetylcodeine, missing








Duplicates in terms of all variables

(0 observations are duplicates)

(file varadd.dta not found)
file varadd.dta saved



    Result                      Number of obs
    -----------------------------------------
    Not matched                             0
    Matched                                20  
    -----------------------------------------


lab_acetylcodeine:


Acetylcodei |
ne detected |
 in PRIMARY |
  abundance |
       only |      Freq.     Percent        Cum.
------------+-----------------------------------
          0 |         19       95.00       95.00
          1 |          1        5.00      100.00
------------+-----------------------------------
      Total |         20      100.00

lab_acetylcodeine_any:


Acetylcodei |
ne detected |
 in primary |
   or trace |
  abundance |      Freq.     Percent        Cum.
------------+-----------------------------------
          0 |         17       85.00       85.00
          1 |          3       15.00      100.0

So there are 3 samples that `lab_acetylcodeine_any`==1 meaning 3 of the samples have acetylcodeine in primary OR trace abundance. Comparing to `lab_acetylcodeine`==1, we see one sample where acetylcodeine abundance was PRIMARY. Let's check this.

In [25]:
frame change lab
list substance abundance lab_acetylcodeine lab_acetylcodeine_any if lab_acetylcodeine_any==1, notrim abbreviate(30) display




           substance   abundance   lab_acetylcodeine   lab_acetylcodeine_any  
 31.   acetylcodeine                               1                       1  
 45.   acetylcodeine       trace                   0                       1  
 60.   acetylcodeine       trace                   0                       1  


Yup, it all checks out. 😄

---

# 8. Abundance and expected
So let's see how many samples fentanyl was in primary and trace abundance.

In [26]:
display "Primary only:"
tabulate lab_fentanyl

display "Primary AND trace abundance:"
tabulate lab_fentanyl_any


Primary only:


   fentanyl |
detected in |
        lab |      Freq.     Percent        Cum.
------------+-----------------------------------
          0 |        105       91.30       91.30
          1 |         10        8.70      100.00
------------+-----------------------------------
      Total |        115      100.00

Primary AND trace abundance:


   fentanyl |
detected in |
        any |
  abundance |      Freq.     Percent        Cum.
------------+-----------------------------------
          0 |        104       90.43       90.43
          1 |         11        9.57      100.00
------------+-----------------------------------
      Total |        115      100.00


So there were 10 samples with fentanyl in primary abundance, one sample where fentanyl was found in trace only. Let's get the `sampleid` for this sample.

In [27]:
tabulate sampleid if lab_fentanyl==0 & lab_fentanyl_any==1


   sampleid |      Freq.     Percent        Cum.
------------+-----------------------------------
        901 |          1      100.00      100.00
------------+-----------------------------------
      Total |          1      100.00


Sampleid 901 contains only fentanyl in trace quantities. Let's look at it more closely.

In [28]:
list sampleid substance abundance if sampleid=="901"


     +---------------------------------------+
     | sampleid         substance   abunda~e |
     |---------------------------------------|
106. |      901          fentanyl      trace |
107. |      901   methamphetamine            |
     +---------------------------------------+


The demo dataset uses 3-digit codes for the sampleid, whereas live data will have 6 digits. Variable `sampleid` is a string because some early IDs contained letters as well.<br>
<br>
Now, let's say we wanted to see what the expected substance was for sample 901. If it were a real sample we could just enter the sampleid on the streetsafe.supply results website. But let's do it with code here. The expected substance information is in the `CARD` dataframe.

In [29]:
// Option 1 - change frames
frame change card
list sampleid expectedsubstance if sampleid=="901"




     +----------------------------+
     | sampleid   expectedsubst~e |
     |----------------------------|
 20. |      901   methamphetamine |
     +----------------------------+


In [30]:
// Option 2 - link frames
** For Stata users you can use -frlink- to link the 2 dataframes and may make more efficient code without having to switch between frames.
** Starting in `LAB` df because the m:1 linkage is easier syntax
frame change lab
frlink m:1 sampleid, frame(card)

** Now we can stay within `LAB` dataframe
frget expectedsubstance, from(card)
drop card // drop auto-generated variable from -frget-
tabulate expectedsubstance if sampleid=="901"



  (all observations in frame lab matched)

  (1 variable copied from linked frame)



    Verbatim expected substance on card |      Freq.     Percent        Cum.
----------------------------------------+-----------------------------------
                        methamphetamine |          2      100.00      100.00
----------------------------------------+-----------------------------------
                                  Total |          2      100.00


Here we get a similar answer but we can see that there are 2 rows in `LAB` dataframe, one for each substance.

In [31]:
list sampleid expectedsubstance substance abundance if sampleid=="901"


     +---------------------------------------------------------+
     | sampleid   expectedsubst~e         substance   abunda~e |
     |---------------------------------------------------------|
106. |      901   methamphetamine          fentanyl      trace |
107. |      901   methamphetamine   methamphetamine            |
     +---------------------------------------------------------+


So it's now the analyst's call if you want to make this an exact match or not between expected and actual. Meth was expected AND detected in primary abundance. That's a positive match. However, fentanyl was not expected but was only found in trace abundance. That's something that would likely be worth letting the participant know about right? It's like a warning to be careful about their supply. In a different scenario where the trace substance was say 4-ANPP and the expected was fentanyl, it is less likely to be important that 4-ANPP wasn't expected since it doesn't have psychoactive properties.